In [232]:
import pandas as pd
import numpy as np


In [233]:
tsv_file='/content/labeledTrainData.tsv'
csv_table=pd.read_table(tsv_file,sep='\t')
csv_table.to_csv('new_name.csv',index=False)

In [234]:
data=pd.read_csv('new_name.csv')

In [235]:
data.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [236]:
data.tail()

,id,sentiment,review
24995,3453_3,0,It seems like more consideration has gone into...
24996,5064_1,0,I don't believe they made this film. Completel...
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...
24999,8478_8,1,I saw this movie as a child and it broke my he...


In [237]:
data.shape

(25000, 3)

In [238]:
data.info

<bound method DataFrame.info of             id  sentiment                                             review
0       5814_8          1  With all this stuff going down at the moment w...
1       2381_9          1  \The Classic War of the Worlds\" by Timothy Hi...
2       7759_3          0  The film starts with a manager (Nicholas Bell)...
3       3630_4          0  It must be assumed that those who praised this...
4       9495_8          1  Superbly trashy and wondrously unpretentious 8...
...        ...        ...                                                ...
24995   3453_3          0  It seems like more consideration has gone into...
24996   5064_1          0  I don't believe they made this film. Completel...
24997  10905_3          0  Guy is a loser. Can't get girls, needs to buil...
24998  10194_3          0  This 30 minute documentary Buñuel made in the ...
24999   8478_8          1  I saw this movie as a child and it broke my he...

[25000 rows x 3 columns]>

In [239]:
data.isna().sum()

id           0
sentiment    0
review       0
dtype: int64

In [240]:
data['sentiment'].nunique()

2

In [241]:
data.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [242]:
from bs4 import BeautifulSoup
import re

In [243]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

In [244]:
data['review'] = data['review'].apply(denoise_text)

<ipython-input-243-4ed251b70956>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [245]:
data['review']

0        With all this stuff going down at the moment w...
1        \The Classic War of the Worlds\" by Timothy Hi...
2        The film starts with a manager (Nicholas Bell)...
3        It must be assumed that those who praised this...
4        Superbly trashy and wondrously unpretentious 8...
                               ...                        
24995    It seems like more consideration has gone into...
24996    I don't believe they made this film. Completel...
24997    Guy is a loser. Can't get girls, needs to buil...
24998    This 30 minute documentary Buñuel made in the ...
24999    I saw this movie as a child and it broke my he...
Name: review, Length: 25000, dtype: object

In [246]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\^^", "", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text

In [247]:
#Apply function on review column
data['review'] = data['review'].apply(remove_special_characters)

In [248]:
data['review']

0        With all this stuff going down at the moment w...
1         The Classic War of the Worlds by Timothy Hine...
2        The film starts with a manager Nicholas Bell g...
3        It must be assumed that those who praised this...
4        Superbly trashy and wondrously unpretentious 8...
                               ...                        
24995    It seems like more consideration has gone into...
24996    I dont believe they made this film Completely ...
24997    Guy is a loser Cant get girls needs to build u...
24998    This 30 minute documentary Buuel made in the e...
24999    I saw this movie as a child and it broke my he...
Name: review, Length: 25000, dtype: object

In [249]:
#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text

In [250]:
#Apply function on review column
data['review']= data['review'].apply(simple_stemmer)

REMOVING STOPWORDS and TOKENIZATION

In [251]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [252]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

{'own', 'as', 'having', "hadn't", 'this', "don't", 'nor', "you'll", 'is', "wasn't", 'into', 'wouldn', 'we', 'its', 'with', 'them', 'm', 'himself', 'all', "you're", 'both', 'did', 'doing', "aren't", 'then', 'mightn', 'yourself', "shouldn't", 'below', 'just', 'there', "wouldn't", 'and', 'of', 'for', 'against', 'my', 'off', "mustn't", 'shouldn', 'their', 'ours', 'most', 've', "mightn't", 'that', 'because', 'those', 't', 'haven', 'hasn', 'o', 'yourselves', "doesn't", 'they', 'up', 'in', 'if', 'herself', 'shan', 'during', 'where', 'few', 're', 'i', 'once', 'are', 'were', 'be', 'she', 'over', 'can', 'now', "weren't", 'until', "couldn't", 'yours', 'theirs', "hasn't", 'what', 'about', 'again', "that'll", "shan't", 'needn', 'ourselves', 'more', 'through', 'only', "didn't", 'out', 'these', "won't", 'has', "it's", 'was', 'had', 'don', 'y', 'other', 'will', 'his', 'at', 'while', 'so', 'themselves', 'ain', 'whom', 'am', 'before', 'aren', "you'd", 'how', 'been', 'your', 'further', 'above', 'me', 'wh

In [253]:
from nltk.tokenize.toktok import ToktokTokenizer

In [254]:
#Tokenization of text
tokenizer1=ToktokTokenizer()

In [255]:
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [256]:
#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer1.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [257]:

data['review'] = data['review'].apply(remove_stopwords)

In [258]:
data['review']

0        thi stuff go moment mj ive start listen hi mus...
1        classic war world timothi hine veri entertain ...
2        film start manag nichola bell give welcom inve...
3        must assum prais thi film greatest film opera ...
4        superbl trashi wondrous unpretenti 80 exploit ...
                               ...                        
24995    seem like consider ha gone imdb review thi fil...
24996    dont believ made thi film complet unnecessari ...
24997    guy loser cant get girl need build pick strong...
24998    thi 30 minut documentari buuel made earli 1930...
24999    saw thi movi child broke heart stori unfinish ...
Name: review, Length: 25000, dtype: object

In [259]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [260]:

data['review']= data['review'].apply(simple_stemmer)

In [261]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(data['review'])
y =  data['sentiment']

In [262]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split



X_train, X_val, y_train, y_val = train_test_split(
    x, y, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:

    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s"
           % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy for C=0.01: 0.80704
Accuracy for C=0.05: 0.84656
Accuracy for C=0.25: 0.87712
Accuracy for C=0.5: 0.88464
Accuracy for C=1: 0.8904


In [263]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
ngram_vectorizer.fit(data['review'])
x = ngram_vectorizer.transform(data['review'])
#y = ngram_vectorizer.transform(data)

X_train, X_val, y_train, y_val = train_test_split(
    x,y, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:

    svm = LinearSVC(C=c)
    svm.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s"
           % (c, accuracy_score(y_val, svm.predict(X_val))))

Accuracy for C=0.01: 0.87872
Accuracy for C=0.05: 0.87872
Accuracy for C=0.25: 0.87872
Accuracy for C=0.5: 0.87824
Accuracy for C=1: 0.87776


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


WORD EMBEDDINGS

In [264]:
globe_path = "glove.6B.300d.txt"

In [265]:
from gensim.models import Word2Vec

In [268]:
preprocess_data=data['review']

In [269]:
cbow_w2v_model = Word2Vec(preprocess_data, vector_size = 300, window = 9, min_count = 2, sg = 0)

In [270]:
def get_embedding_w2v(doc_tokens, pre_trained):
    embeddings = []

    model = cbow_w2v_model
    for tok in doc_tokens:
            if tok in model.wv.index_to_key:
                embeddings.append(model.wv.get_vector(tok))
    # iterate over tokens to extract their vectors

    # mean the vectors of individual words to get the vector of the statement
    return np.mean(embeddings, axis=0)

In [ ]:
#preprocess_data=data['review']

In [271]:
from sklearn.naive_bayes import GaussianNB

In [272]:
X_w2v_model = preprocess_data.apply(lambda x: get_embedding_w2v(x, pre_trained = 0))

In [273]:
X_w2v_model = pd.DataFrame(X_w2v_model.tolist())

In [274]:
y=data['sentiment']

In [275]:
x_train, x_test, y_train, y_test = train_test_split(X_w2v_model , y, test_size = 0.2, random_state=42)

In [276]:
model1 = GaussianNB()
model1.fit(x_train, y_train)
y_pred = model1.predict(x_test)
accuracy_score(y_test, y_pred)

0.592

In [277]:
X_w2v_pt_model = preprocess_data.apply(lambda x: get_embedding_w2v(x, pre_trained = 1))

In [278]:
X_w2v_pt_model = pd.DataFrame(X_w2v_pt_model.tolist())

In [279]:
x_train, x_test, y_train, y_test = train_test_split(X_w2v_pt_model , y, test_size = 0.2, random_state=42)

In [281]:
model2 = GaussianNB()
model2.fit(x_train, y_train)
y_pred = model2.predict(x_test)
accuracy_score(y_test, y_pred)

0.592